# Capstone Project - Explore the optimal location for a residence in Bangalore
Table of contents
1.	Introduction
2.	Data
3.	Methodology
4.	Analysis
5.	Results and Discussion
6.	Conclusion

## 1.	Introduction

### 1.1	Background

Bangalore, officially known as Bengaluru is the capital of the Indian state of Karnataka. It has a population of over ten million. making it a megacity and the third-most populous city and fifth-most populous urban agglomeration in India. It is located in southern India.

Bengaluru is widely regarded as the "Silicon Valley of India" (or "IT capital of India") because of its role as the nation's leading information technology (IT) exporter. Indian technological organizations such as ISRO, Infosys, Wipro and HAL are headquartered in the city. A demographically diverse city, Bangalore is the second fastest-growing major metropolis in India. Recent estimates of the metro economy of its urban area have ranked Bangalore either the fourth or fifth-most productive metro area of India

Day by day the population of the city is increasing and there are more visitors to the city. As it’s considered as one of the biggest IT hub in India, Due to which a lot of people migrating here.

This leads to a good business opportunity. There is ample scope for building Residential Apartments /investing on property. 

### 1.2	Business Problem

In this project we will try to find a good location/Residential Apartment for a residence in Bangalore city.  There are many residential places in the city. But we would like to identify an ideal location where you get all the basic amenities like (restaurants, schools, colleges, bus stop, shopping malls, pharmacy and Hospitals) which are nearby to the location.

Residential Apartment/location having nearby restaurants, schools, colleges, bus stop, shopping malls, pharmacy and Hospitals are normally considered as an ideal place. should propose an optimal location, that should be within the 1kms (ideal distance) from the Residence.


In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd
import requests
import math

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!pip install geopy
from geopy.geocoders import Nominatim 

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

import logging

print('Libraries imported.')

     |████████████████████████████████| 102kB 9.1MB/s ta 0:00:011
Libraries imported.




Lets get the latitude and longitude of Bangalore city usi geopy library.

In [3]:
# get the address of Banglore city
address = 'Banglore, Karnataka, India'

# get geolocator
geolocator = Nominatim(user_agent="ny_explorer")

# get location from address
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Banglore City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Banglore City are 12.9587464, 77.5573456.




Foursquare API is the main API source with which we get all the top locations for various categories and then the nearby residences for each such location. We use venues/search API for category specific search based on latitude and longitude. We get the category id for all the categories from Foursquare API documentation.

In [4]:
#@title Foursquare details

CLIENT_ID = 'MIBUDS0IYOS2BPOZVEU34K3FXLHGCSQ0N0AIZ43SHZC1UJL1' # your Foursquare ID
CLIENT_SECRET = 'Y4OPOIH4BQR4IBXJDRJLCOS134MXX2GLUSCYGTWRQROWFBHR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version



Now, define a function to get the top 100 venues that are in Bangalore within a radius of 10kms based on category id.

let's use Foursquare API to get info on residences in each category locations.

We're interested in venues in 'residence' category, for each category location.

In [5]:
radius = 10000
LIMIT = 100

def get_Url(categoryId):
    return 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId)

In [6]:
def get_nearby_venues(results):
    #logging.debug("result is %s",str(results))
    #print(results)

    # clean the data
    venues = results['response']['venues']
    #response = results['response']

    nearby_venues = json_normalize(venues) # flatten JSON

    if nearby_venues.shape[0] > 0:
        # filter columns
        filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
        nearby_venues = nearby_venues.loc[:, filtered_columns]

        # filter the category for each row
        nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

        # clean columns
        nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

    return nearby_venues

Define utility functions to extract the venues for a category

In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
# get the nearby venues based on the category id in Banglore within 20KM radius
def get_nearby_venues_for_category(categoryId):
    # first get the url
    url = get_Url(categoryId)

    # get the json response from url
    results = requests.get(url).json()

    # return the venues for the provided category
    return get_nearby_venues(results)

Define the category id for categories -Residence, Restaurants, schools, colleges, bus stop,shopping malls, pharmacy and Hospitals

In [9]:
# Foursquare category codes
category_Residence = "4d954b06a243a5684965b473"
category_food = "4d4b7105d754a06374d81259"
category_Bus_Stop = "52f2ab2ebcbc57f1066b8b4f"
category_schools = "4bf58dd8d48988d13b941735"
category_colleges = "4d4b7105d754a06372d81259"
category_hospitals = "4bf58dd8d48988d196941735"
category_Pharmacy = "4bf58dd8d48988d10f951735"
category_Shopping_Mall = "4bf58dd8d48988d1fd941735"


Lets first get the top 100 Restaurants within Bangalore city 10km radius. Also, clean the data,
Finally display the Restaurants list.

In [10]:
# get the food list

food = get_nearby_venues_for_category(category_food)

#clean the food data based on the category containing only Restaurant
food = food[food['categories'].str.contains("Restaurant")].reset_index(drop=True)
food


,name,categories,lat,lng
0,Brahmins Coffee Bar,South Indian Restaurant,12.943772,77.560114
1,7 PLATES,Halal Restaurant,12.997218,77.603940
2,Madeena Hotel,Indian Restaurant,12.935153,77.615925
3,Nagarjuna,Andhra Restaurant,12.904843,77.601733
4,Sukh Sagar,Vegetarian / Vegan Restaurant,12.933471,77.623099
5,Priyadarshini Grand,Fast Food Restaurant,12.990235,77.538020
6,Sagar Fast Food,Fast Food Restaurant,12.996623,77.554179
7,Nagarjuna,Andhra Restaurant,12.933693,77.621156
8,Eden Park Restaurant,Indian Restaurant,12.983456,77.598707
9,Kabab Magic,Indian Restaurant,12.916248,77.596899


Lets first get the top 100 colleges within Bangalore city 10 km radius. Also, clean the data which is having University in the category,
Finally display the colleges list.

In [11]:
# get the colleges list
colleges = get_nearby_venues_for_category(category_colleges)

#clean the colleges data based on the category containing only University
colleges = colleges[colleges['categories'].str.contains("University" )].reset_index(drop=True)
colleges

,name,categories,lat,lng
0,Karnataka Samskrit University,University,12.959672,77.571990
1,Christ University,University,12.935617,77.606306
2,Indian Institute Of Science,University,13.019109,77.567425
3,Sri Bhagwan Mahaveer Jain College,College & University,12.918222,77.591793
4,M S Ramaiah Institute of Technology Bangalore,General College & University,13.031549,77.564044
5,J T S Institute,General College & University,12.981077,77.601791
6,"National Public School, Koramangala",General College & University,12.946111,77.622985
7,Bangalore University,University,12.947261,77.503470
8,R V Teacher's College,General College & University,12.943335,77.580963
9,Rajiv Gandhi Institute of Technology,General College & University,13.032886,77.597938


Lets first get the top 100 Shopping_Mall within Bangalore city 10 km radius. Also, clean the data which is having Shopping Mall in the category,
Finally display the Shopping_Mall list.

In [12]:
# get the Shopping_Mall list
Shopping_Mall = get_nearby_venues_for_category(category_Shopping_Mall)

#clean the Shopping_Mall data based on the category containing only Shopping_Mall
Shopping_Mall = Shopping_Mall[Shopping_Mall['categories'].str.contains("Shopping Mall")].reset_index(drop=True)
Shopping_Mall

,name,categories,lat,lng
0,ETA Mall,Shopping Mall,12.965804,77.561811
1,Indo Dubai Plaza,Shopping Mall,12.973871,77.606947
2,Safina Plaza,Shopping Mall,12.979554,77.607022
3,Village Supermarket,Shopping Mall,12.999546,77.622963
4,HSR BDA Complex,Shopping Mall,12.913092,77.637377
5,BDA Shopping Complex,Shopping Mall,12.930855,77.622548
6,Big Market,Shopping Mall,12.897157,77.585988
7,UB City,Shopping Mall,12.971709,77.595905
8,Vega City Mall,Shopping Mall,12.907603,77.601140
9,Lifestyle,Shopping Mall,12.966934,77.611553


Lets first get the top 100 Bus_Stop within Bangalore city 10 km radius. Also, clean the data.
Finally display the Bus_Stop list.

In [13]:
# get the nearby Bus_Stop
Bus_Stop = get_nearby_venues_for_category(category_Bus_Stop)

#display the Bus_Stops 
Bus_Stop

,name,categories,lat,lng
0,Satellite Bus Stop Mysore Road,Bus Stop,12.953278,77.543330
1,Jeedimaara,Bus Stop,12.907615,77.600471
2,Sangam Circle,Bus Stop,12.917545,77.578609
3,Indian Express Bus Stop,Bus Stop,12.983797,77.596576
4,CMH Hospital Bus Stop,Bus Stop,12.978642,77.645746
5,thumkur,Bus Stop,12.989453,77.542221
6,durgama motors,Bus Stop,12.935039,77.624354
7,Vijayanagar Bus Stop,Bus Stop,12.971844,77.537672
8,Doopanahalli Bus Stop,Bus Stop,12.956775,77.649325
9,Central Bus Stop,Bus Stop,13.006866,77.554472


Lets first get the top 100 Pharmacy within Bangalore city 10 km radius. Also, clean the data which is having Pharmacy in the category,.
Finally display the Pharmacy list.

In [14]:
# get the nearby Pharmacy
Pharmacy = get_nearby_venues_for_category(category_Pharmacy)

#clean the Pharmacy data based on the category containing only Pharmacy
Pharmacy = Pharmacy[Pharmacy['categories'].str.contains("Pharmacy")].reset_index(drop=True)
Pharmacy

,name,categories,lat,lng
0,Apollo Pharmacy,Pharmacy,12.936331,77.614682
1,Acura Pharma,Pharmacy,12.935310,77.620490
2,Ganesh Pharma,Pharmacy,12.886212,77.590488
3,Sanjeevini Pharma,Pharmacy,12.983333,77.583333
4,ClickOnCare.com Office,Pharmacy,12.924233,77.610588
5,Sri Vinayaka Medicals,Pharmacy,12.970200,77.650795
6,Galaxy Swathi Pharmacy,Pharmacy,12.909582,77.564568
7,Fortis Healthworld,Pharmacy,12.959034,77.522957
8,R. A. A. W. D,Pharmacy,12.959733,77.649544
9,sudhaparimala,Pharmacy,12.934591,77.589872


Lets first get the top 100 schools within Bangalore city 10 km radius. Also, clean the data which is having High School in the category.
Finally display the School list.

In [15]:
# get the nearby schools
schools = get_nearby_venues_for_category(category_schools)

# filter the schools data based on category - High School
schools = schools[schools['categories'] == 'High School'].reset_index(drop=True)

schools

,name,categories,lat,lng
0,IQRA International School,High School,12.917759,77.614330
1,st charles high school,High School,13.007509,77.636220
2,Goodwill Girls High School,High School,12.995331,77.609204
3,Siddhartha High School,High School,13.029993,77.592642
4,KLE international school,High School,12.996765,77.553414
5,achieve school of edcation,High School,12.954518,77.573753
6,Vidyanext Tuition Classes,High School,12.985690,77.542498
7,Mary Immaculate School,High School,12.945718,77.599380
8,St. Joseph's Boys' High School,High School,12.972003,77.603593
9,Brigade school @ Brigade Gateway,High School,13.012246,77.557560


Lets first get the top 100 hospitals within Bangalore city 10 km radius. Also, clean the data which is having hospital in the category,
Finally display the hospitals list.

In [16]:
# get the hospitals list
hospitals = get_nearby_venues_for_category(category_hospitals)

# filter the hospitals data based on category - Hospital
hospitals = hospitals[hospitals['categories'] == 'Hospital'].reset_index(drop=True)

hospitals

,name,categories,lat,lng
0,Shankara Cancer Hospital,Hospital,12.954407,77.571001
1,Sagar Chandramma Hospital,Hospital,12.948401,77.574827
2,People Tree Hospitals @Ragavendra,Hospital,13.042530,77.515540
3,Manipal Hospital,Hospital,12.959238,77.649208
4,HCG Oncology,Hospital,12.964278,77.589562
5,Columbia Asia Hospital,Hospital,13.050844,77.593750
6,Sagar Hospital,Hospital,12.928226,77.599922
7,St. Johns Medical College,Hospital,12.929369,77.618436
8,M.S Ramaiah Memorial Hospital,Hospital,13.028492,77.569780
9,Chinmaya Mission Hospital,Hospital,12.977870,77.645956


Lets first get the top 100 Residence  within Bangalore city 10 km radius. Also, clean the data.
Finally display the Residence list.

In [17]:
# get the nearby category_Residence  
Residence = get_nearby_venues_for_category(category_Residence)
Residence

,name,categories,lat,lng
0,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188
1,Ansal Krisna,Residential Building (Apartment / Condo),12.937972,77.610031
2,Victory Ashoka Enclave,Residential Building (Apartment / Condo),13.000334,77.611456
3,Arya Vishwas,Residential Building (Apartment / Condo),12.941284,77.557651
4,Casa Nuestra,Residential Building (Apartment / Condo),12.878629,77.579239
5,Elita Promenade,Residential Building (Apartment / Condo),12.893116,77.579782
6,6th Block Rajaji Nagar,Residential Building (Apartment / Condo),12.979021,77.554819
7,Salarpuria Greenage,Residential Building (Apartment / Condo),12.901186,77.629517
8,Brigade Gateway,Residential Building (Apartment / Condo),13.012168,77.556288
9,"Brigade Gardenia, JP Nagar 7th Phase",Residential Building (Apartment / Condo),12.886564,77.578361


Lets plot these top spots on the map centered at Bangalore city. Each location is shown in vaious color marker

In [18]:
# create map of Bangalore using latitude and longitude values
map_Bangalore = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
  
for lat, lng, category, name in zip(food['lat'], food['lng'], food['categories'], food['name']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Bangalore) 

for lat, lng, category, name in zip(schools['lat'], schools['lng'], schools['categories'], schools['name']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Bangalore) 
    
for lat, lng, category, name in zip(hospitals['lat'], hospitals['lng'], hospitals['categories'], hospitals['name']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Bangalore) 

for lat, lng, category, name in zip(Bus_Stop['lat'], Bus_Stop['lng'], Bus_Stop['categories'], Bus_Stop['name']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='orange',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Bangalore) 
    
for lat, lng, category, name in zip(Pharmacy['lat'], Pharmacy['lng'], Pharmacy['categories'], Pharmacy['name']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Bangalore) 
		
for lat, lng, category, name in zip(Shopping_Mall['lat'], Shopping_Mall['lng'], Shopping_Mall['categories'], Shopping_Mall['name']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Bangalore) 

for lat, lng, category, name in zip(colleges['lat'], colleges['lng'], colleges['categories'], colleges['name']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='pink',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Bangalore) 
    
map_Bangalore

#### 3.	Methodology
In this project we will put our efforts on identifying the optimal location for a residence in Bangalore city within 10km range.

We first identified the categories which are usually basis necessity will people consider before purchasing or investing on property. Those are restaurants, schools, colleges, bus stop,shopping malls, pharmacy and Hospitals. These 8 categories are much necessary compared to other categories such as Art, Museum, Pool etc.

1.	In the first step we have collected locations data for different categories.Details for each location includes name, latitude, longitude and category name. Plot these locations on the map with different colors.
2.	Second step in our analysis will be getting the nearby residences within 1km distance from each of these locations. We feel 1km is a reasonable distance from any location. Then plotting these locations on the map to display the nearby residences along with locations.
3.	Third step is to analyze the consolidated data which we obtained in step 2. This analysis will shed light on the recommended locations.
4.	Final step is to recommend the optimal location to the stakeholders.


In [19]:
# set the radius and limit
radius_s = 1000 # 1 kms range  
LIMIT_s = 5 # top 5 nearby schools
 
# function to get nearby schools for a spot
def get_nearby_schools_for_a_spot(latitude, longitude):

    spot_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius_s, LIMIT_s, category_schools)

    results = requests.get(spot_url).json()

    # return the venues for the provided category
    return get_nearby_venues(results)

In [20]:
# for each spot in Residence get the nearby schools within 1km
column_names = ["location", "location_category", "location_lat", "location_lng", "name", "categories", "lat", "lng"]

schools_combined = pd.DataFrame(columns = column_names)
df = Residence
for lat, lng, name, category in zip(df['lat'], df['lng'], df['name'], df['categories']):
    # get the nearby schools for a spot
    schools = get_nearby_schools_for_a_spot(lat, lng)

    if schools.shape[0] > 0:
        # filter based on the category value containing High School
        schools = schools[schools['categories'].str.contains("High School")].reset_index(drop=True)
        schools.insert(0, 'location', name)
        schools.insert(1, 'location_category', category)
        schools.insert(2, 'location_lat', lat)
        schools.insert(3, 'location_lng', lng)
        
        if schools.shape[0] == 1 :
            print("Only one School found near Residence:", name)
        if schools.shape[0] == 2 :
            print("Only two School found near Residence:", name)
        if schools.shape[0] == 3 :
            print("Only three School found near Residence:", name)
        if schools.shape[0] == 4 :
            print("Only four School found near Residence:", name)
        if schools.shape[0] > 4 :
            print("More than four School found near Residence:", name)

        # concat the schools data to the combined dataframe
        schools_combined = pd.concat([schools_combined, schools])
    else:
        print("NO School found within 1 KMS range near Residence:", name)

schools_combined.reset_index(drop=True)
schools_combined

Only one School found near Residence: Diamond District
Only one School found near Residence: Victory Ashoka Enclave
Only one School found near Residence: Arya Vishwas
Only one School found near Residence: Elita Promenade
Only two School found near Residence: 6th Block Rajaji Nagar
Only three School found near Residence: Brigade Gateway
Only one School found near Residence: Brigade Gardenia, JP Nagar 7th Phase
Only one School found near Residence: Pebble Bay
Only three School found near Residence: HM Winchester
Only one School found near Residence: Trivandrum Lodge
Only one School found near Residence: Sobha Dew Flower
Only one School found near Residence: Ali Apartments
NO School found within 1 KMS range near Residence: Mahakali Layout
Only three School found near Residence: 11th Cross First Block Rajajinagar
NO School found within 1 KMS range near Residence: HM World City
Only one School found near Residence: MSR Nagar
Only two School found near Residence: Esteem Park Apartments
Only 

,location,location_category,location_lat,location_lng,name,categories,lat,lng
0,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,National Public School,High School,12.968772,77.646713
0,Victory Ashoka Enclave,Residential Building (Apartment / Condo),13.000334,77.611456,Goodwill Girls High School,High School,12.995331,77.609204
0,Arya Vishwas,Residential Building (Apartment / Condo),12.941284,77.557651,Sree kumaraswamy vidya mandir high school,High School,12.942284,77.563518
0,Elita Promenade,Residential Building (Apartment / Condo),12.893116,77.579782,Jyothi Kendriya Vidyalya,High School,12.895696,77.570637
0,6th Block Rajaji Nagar,Residential Building (Apartment / Condo),12.979021,77.554819,St. Miras High School,High School,12.977707,77.555664
1,6th Block Rajaji Nagar,Residential Building (Apartment / Condo),12.979021,77.554819,Vanitha Seva Mandir English School,High School,12.986713,77.558868
0,Brigade Gateway,Residential Building (Apartment / Condo),13.012168,77.556288,Brigade school @ Brigade Gateway,High School,13.012246,77.557560
1,Brigade Gateway,Residential Building (Apartment / Condo),13.012168,77.556288,Sardar Patel High School,High School,13.003457,77.552350
2,Brigade Gateway,Residential Building (Apartment / Condo),13.012168,77.556288,East West Public School,High School,13.005373,77.556929
0,"Brigade Gardenia, JP Nagar 7th Phase",Residential Building (Apartment / Condo),12.886564,77.578361,Brigade School @ JP Nagar,High School,12.891879,77.585289


In [21]:
# set the radius and limit
radius_h = 1000 # 1 kms range 
LIMIT_h = 5 # top 5 nearby hospitals
 
# function to get nearby hospitals for a spot
def get_nearby_hospitals_for_a_spot(latitude, longitude):

    spot_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius_h, LIMIT_h, category_hospitals)

    results = requests.get(spot_url).json()

    # return the venues for the provided category
    return get_nearby_venues(results)

In [22]:
# for each spot in Residence get the nearby hospitals within 1km
column_names = ["location", "location_category", "location_lat", "location_lng", "name", "categories", "lat", "lng"]

hospitals_combined = pd.DataFrame(columns = column_names)
df = Residence
for lat, lng, name, category in zip(df['lat'], df['lng'], df['name'], df['categories']):
    # get the nearby hospitals for a spot
    hospitals = get_nearby_hospitals_for_a_spot(lat, lng)

    if hospitals.shape[0] > 0:
        # filter based on the category value containing Hospital
        hospitals = hospitals[hospitals['categories'].str.contains("Hospital")].reset_index(drop=True)
        hospitals.insert(0, 'location', name)
        hospitals.insert(1, 'location_category', category)
        hospitals.insert(2, 'location_lat', lat)
        hospitals.insert(3, 'location_lng', lng)
        
        if hospitals.shape[0] == 1 :
            print("Only one hospital found near Residence:", name)
        if hospitals.shape[0] == 2 :
            print("Only two hospitals found near Residence:", name)
        if hospitals.shape[0] == 3 :
            print("Only three hospitals found near Residence:", name)
        if hospitals.shape[0] == 4 :
            print("Only four hospitals found near Residence:", name)
        if hospitals.shape[0] > 4 :
            print("More than four hospitals found near Residence:", name)

        # concat the hospitals data to the combined dataframe
        hospitals_combined = pd.concat([hospitals_combined, hospitals])
    else:
        print("NO hospitals found within 1 KMS range near Residence:", name)

hospitals_combined.reset_index(drop=True)
hospitals_combined

More than four hospitals found near Residence: Diamond District
More than four hospitals found near Residence: Ansal Krisna
Only four hospitals found near Residence: Victory Ashoka Enclave
More than four hospitals found near Residence: Arya Vishwas
Only two hospitals found near Residence: Casa Nuestra
Only four hospitals found near Residence: Elita Promenade
More than four hospitals found near Residence: 6th Block Rajaji Nagar
Only four hospitals found near Residence: Salarpuria Greenage
More than four hospitals found near Residence: Brigade Gateway
Only four hospitals found near Residence: Brigade Gardenia, JP Nagar 7th Phase
More than four hospitals found near Residence: Pebble Bay
Only three hospitals found near Residence: L & T South city
Only three hospitals found near Residence: HM Winchester
Only three hospitals found near Residence: Vars Topaz Apartments
Only four hospitals found near Residence: Trivandrum Lodge
More than four hospitals found near Residence: Sobha Dew Flower
On

,location,location_category,location_lat,location_lng,name,categories,lat,lng
0,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,Manipal Hospital,Hospital,12.959238,77.649208
1,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,The Apollo Clinic,Hospital,12.965529,77.641534
2,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,Manipal Health Check Reception,Hospital Ward,12.958470,77.649058
3,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,Commondo hospital,Hospital,12.962708,77.635264
4,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,Radology Department,Hospital,12.958344,77.648940
0,Ansal Krisna,Residential Building (Apartment / Condo),12.937972,77.610031,Prasad Eye Hospital,Hospital,12.934667,77.609482
1,Ansal Krisna,Residential Building (Apartment / Condo),12.937972,77.610031,pristine consultation and diagnostics,Hospital,12.938283,77.618655
2,Ansal Krisna,Residential Building (Apartment / Condo),12.937972,77.610031,St John's Hospital,Hospital,12.929770,77.618554
3,Ansal Krisna,Residential Building (Apartment / Condo),12.937972,77.610031,Pediatric Ward-St.johns Medical College Hospital,Hospital,12.929897,77.616534
4,Ansal Krisna,Residential Building (Apartment / Condo),12.937972,77.610031,St. Johns Medical College,Hospital,12.929369,77.618436


In [23]:
# set the radius and limit
radius_f = 1000 # 1 kms range 
LIMIT_f = 5 # top 5 nearby schools
 
# function to get nearby Restaurant for a spot    
def get_nearby_food_for_a_spot(latitude, longitude):

    spot_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius_f, LIMIT_f, category_food)

    results = requests.get(spot_url).json()

    # return the venues for the provided category
    return get_nearby_venues(results)

In [24]:
# for each spot in Residence get the nearby schools within 1km
column_names = ["location", "location_category", "location_lat", "location_lng", "name", "categories", "lat", "lng"]

food_combined = pd.DataFrame(columns = column_names)
df = Residence
for lat, lng, name, category in zip(df['lat'], df['lng'], df['name'], df['categories']):
    # get the nearby food for a spot
    food = get_nearby_food_for_a_spot(lat, lng)

    if food.shape[0] > 0:
        # filter based on the category value containing Restaurant
        food = food[food['categories'].str.contains("Restaurant")].reset_index(drop=True)
        food.insert(0, 'location', name)
        food.insert(1, 'location_category', category)
        food.insert(2, 'location_lat', lat)
        food.insert(3, 'location_lng', lng)
        
        if food.shape[0] == 1 :
            print("Only one Restraunt found near Residence:", name)
        if food.shape[0] == 2 :
            print("Only two Restraunt found near Residence:", name)
        if food.shape[0] == 3 :
            print("Only three Restraunt found near Residence:", name)
        if food.shape[0] == 4 :
            print("Only four Restraunt found near Residence:", name)
        if food.shape[0] > 4 :
            print("More than four Restraunt found near Residence:", name)

        # concat the restaurants data to the combined dataframe
        food_combined = pd.concat([food_combined, food])
    else:
        print("NO Restraunt found within 1 KMS range near Residence:", name)

food_combined.reset_index(drop=True)
food_combined

Only two Restraunt found near Residence: Diamond District
Only one Restraunt found near Residence: Ansal Krisna
Only three Restraunt found near Residence: Victory Ashoka Enclave
Only two Restraunt found near Residence: Arya Vishwas
Only three Restraunt found near Residence: Casa Nuestra
Only three Restraunt found near Residence: Elita Promenade
Only two Restraunt found near Residence: 6th Block Rajaji Nagar
Only three Restraunt found near Residence: Salarpuria Greenage
Only one Restraunt found near Residence: Brigade Gateway
Only four Restraunt found near Residence: Brigade Gardenia, JP Nagar 7th Phase
Only four Restraunt found near Residence: Pebble Bay
Only two Restraunt found near Residence: L & T South city
Only two Restraunt found near Residence: HM Winchester
Only three Restraunt found near Residence: Vars Topaz Apartments
Only three Restraunt found near Residence: Trivandrum Lodge
Only two Restraunt found near Residence: Sobha Dew Flower
Only two Restraunt found near Residence: 

,location,location_category,location_lat,location_lng,name,categories,lat,lng
0,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,Flax Healthy Living,Restaurant,12.967861,77.648125
1,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,Sri Udupi Food Hub,Vegetarian / Vegan Restaurant,12.966426,77.637402
0,Ansal Krisna,Residential Building (Apartment / Condo),12.937972,77.610031,Madeena Hotel,Indian Restaurant,12.935153,77.615925
0,Victory Ashoka Enclave,Residential Building (Apartment / Condo),13.000334,77.611456,Rahhams Restaurant,Indian Restaurant,13.000867,77.613115
1,Victory Ashoka Enclave,Residential Building (Apartment / Condo),13.000334,77.611456,7 PLATES,Halal Restaurant,12.997218,77.603940
2,Victory Ashoka Enclave,Residential Building (Apartment / Condo),13.000334,77.611456,Sri Devi Cafe,Vegetarian / Vegan Restaurant,12.997773,77.620439
0,Arya Vishwas,Residential Building (Apartment / Condo),12.941284,77.557651,Brahmins Coffee Bar,South Indian Restaurant,12.943772,77.560114
1,Arya Vishwas,Residential Building (Apartment / Condo),12.941284,77.557651,Kamat Bugle Rock,Indian Restaurant,12.939907,77.566755
0,Casa Nuestra,Residential Building (Apartment / Condo),12.878629,77.579239,Brundavana Pure Veg,Vegetarian / Vegan Restaurant,12.872851,77.583446
1,Casa Nuestra,Residential Building (Apartment / Condo),12.878629,77.579239,bhatru chats,Indian Restaurant,12.883639,77.570107


In [25]:
# set the radius and limit
radius_sm = 1000 # 1 kms range 
LIMIT_sm = 5 # top 5 nearby Shopping_Mall

# function to get nearby Shopping_Mall for a spot  
def get_nearby_Shopping_Mall_for_a_spot(latitude, longitude):

    spot_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius_sm, LIMIT_sm, category_Shopping_Mall)

    results = requests.get(spot_url).json()

    # return the venues for the provided category
    return get_nearby_venues(results)


In [26]:
# for each spot in Residence get the nearby Shopping_Mall  within 1km
column_names = ["location", "location_category", "location_lat", "location_lng", "name", "categories", "lat", "lng"]

Shopping_Mall_combined = pd.DataFrame(columns = column_names)
df = Residence
for lat, lng, name, category in zip(df['lat'], df['lng'], df['name'], df['categories']):
    # get the nearby Shopping_Mall for a spot
    Shopping_Mall = get_nearby_Shopping_Mall_for_a_spot(lat, lng)

    if Shopping_Mall.shape[0] > 0:
        # filter based on the category value 
        Shopping_Mall.insert(0, 'location', name)
        Shopping_Mall.insert(1, 'location_category', category)
        Shopping_Mall.insert(2, 'location_lat', lat)
        Shopping_Mall.insert(3, 'location_lng', lng)
        
        if Shopping_Mall.shape[0] == 1 :
            print("Only one Shopping_Mall found near Residence:", name)
        if Shopping_Mall.shape[0] == 2 :
            print("Only two Shopping_Mall found near Residence:", name)
        if Shopping_Mall.shape[0] == 3 :
            print("Only three Shopping_Mall found near Residence:", name)
        if Shopping_Mall.shape[0] == 4 :
            print("Only four Shopping_Mall found near Residence:", name)
        if Shopping_Mall.shape[0] > 4 :
            print("More than four Shopping_Mall found near Residence:", name)

        # concat the Shopping_Mall data to the combined dataframe
        Shopping_Mall_combined = pd.concat([Shopping_Mall_combined, Shopping_Mall])
    else:
        print("NO Shopping_Mall found within 1 KMS range near Residence:", name)

Shopping_Mall_combined.reset_index(drop=True)
Shopping_Mall_combined

Only three Shopping_Mall found near Residence: Diamond District
More than four Shopping_Mall found near Residence: Ansal Krisna
NO Shopping_Mall found within 1 KMS range near Residence: Victory Ashoka Enclave
Only two Shopping_Mall found near Residence: Arya Vishwas
NO Shopping_Mall found within 1 KMS range near Residence: Casa Nuestra
Only one Shopping_Mall found near Residence: Elita Promenade
Only four Shopping_Mall found near Residence: 6th Block Rajaji Nagar
Only one Shopping_Mall found near Residence: Salarpuria Greenage
More than four Shopping_Mall found near Residence: Brigade Gateway
NO Shopping_Mall found within 1 KMS range near Residence: Brigade Gardenia, JP Nagar 7th Phase
NO Shopping_Mall found within 1 KMS range near Residence: Pebble Bay
Only two Shopping_Mall found near Residence: L & T South city
More than four Shopping_Mall found near Residence: HM Winchester
Only four Shopping_Mall found near Residence: Vars Topaz Apartments
Only three Shopping_Mall found near Resid

,location,location_category,location_lat,location_lng,name,categories,lat,lng
0,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,The Colonnade,Shopping Mall,12.960689,77.647032
1,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,Leela Galleria,Shopping Mall,12.960112,77.648535
2,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,Kemp Fort Mall,Shopping Mall,12.959733,77.647813
0,Ansal Krisna,Residential Building (Apartment / Condo),12.937972,77.610031,Star Bazaar,Department Store,12.936956,77.610709
1,Ansal Krisna,Residential Building (Apartment / Condo),12.937972,77.610031,Shopper's Stop,Shopping Mall,12.933395,77.612697
2,Ansal Krisna,Residential Building (Apartment / Condo),12.937972,77.610031,Big Bazaar,Miscellaneous Shop,12.933031,77.611998
3,Ansal Krisna,Residential Building (Apartment / Condo),12.937972,77.610031,Tibet Mall,Clothing Store,12.934202,77.616014
4,Ansal Krisna,Residential Building (Apartment / Condo),12.937972,77.610031,The Forum Mall,Shopping Mall,12.934603,77.611312
0,Arya Vishwas,Residential Building (Apartment / Condo),12.941284,77.557651,South India Shopping Mall,Shopping Mall,12.936093,77.563181
1,Arya Vishwas,Residential Building (Apartment / Condo),12.941284,77.557651,gopalan mall,Shopping Mall,12.937538,77.550878


In [28]:
# set the radius and limit
radius_p = 1000 # 1 kms range 
LIMIT_p = 5 # top 5 nearby Pharmacy

# function to get nearby Pharmacy for a spot
def get_nearby_Pharmacy_for_a_spot(latitude, longitude):

    spot_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius_sm, LIMIT_sm, category_Pharmacy)

    results = requests.get(spot_url).json()

    # return the venues for the provided category
    return get_nearby_venues(results)

In [29]:
# for each spot in Residence get the nearby Pharmacy within 1km
column_names = ["location", "location_category", "location_lat", "location_lng", "name", "categories", "lat", "lng"]

Pharmacy_combined = pd.DataFrame(columns = column_names)
df = Residence
for lat, lng, name, category in zip(df['lat'], df['lng'], df['name'], df['categories']):
    # get the nearby Pharmacy for a spot
    Pharmacy = get_nearby_Pharmacy_for_a_spot(lat, lng)

    if Pharmacy.shape[0] > 0:
        Pharmacy.insert(0, 'location', name)
        Pharmacy.insert(1, 'location_category', category)
        Pharmacy.insert(2, 'location_lat', lat)
        Pharmacy.insert(3, 'location_lng', lng)
        
        if Pharmacy.shape[0] == 1 :
            print("Only one Pharmacy found near Residence:", name)
        if Pharmacy.shape[0] == 2 :
            print("Only two Pharmacy found near Residence:", name)
        if Pharmacy.shape[0] == 3 :
            print("Only three Pharmacy found near Residence:", name)
        if Pharmacy.shape[0] == 4 :
            print("Only four Pharmacy found near Residence:", name)
        if Pharmacy.shape[0] > 4 :
            print("More than four Pharmacy found near Residence:", name)

        # concat the Pharmacy data to the combined dataframe
        Pharmacy_combined = pd.concat([Pharmacy_combined, Pharmacy])
    else:
        print("NO Pharmacy found within 1 KMS range near Residence:", name)

Pharmacy_combined.reset_index(drop=True)
Pharmacy_combined

More than four Pharmacy found near Residence: Diamond District
Only four Pharmacy found near Residence: Ansal Krisna
More than four Pharmacy found near Residence: Victory Ashoka Enclave
Only four Pharmacy found near Residence: Arya Vishwas
Only two Pharmacy found near Residence: Casa Nuestra
Only four Pharmacy found near Residence: Elita Promenade
Only three Pharmacy found near Residence: 6th Block Rajaji Nagar
Only one Pharmacy found near Residence: Salarpuria Greenage
Only three Pharmacy found near Residence: Brigade Gateway
Only four Pharmacy found near Residence: Brigade Gardenia, JP Nagar 7th Phase
More than four Pharmacy found near Residence: Pebble Bay
Only four Pharmacy found near Residence: L & T South city
Only four Pharmacy found near Residence: HM Winchester
More than four Pharmacy found near Residence: Vars Topaz Apartments
Only four Pharmacy found near Residence: Trivandrum Lodge
Only three Pharmacy found near Residence: Sobha Dew Flower
Only one Pharmacy found near Resid

,location,location_category,location_lat,location_lng,name,categories,lat,lng
0,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,Manipal Hospital,Hospital,12.959238,77.649208
1,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,Religarewellness 1,Pharmacy,12.966646,77.637350
2,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,Religare,Pharmacy,12.959603,77.653291
3,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,Ram Medicals,Pharmacy,12.958814,77.638428
4,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,R. A. A. W. D,Pharmacy,12.959733,77.649544
0,Ansal Krisna,Residential Building (Apartment / Condo),12.937972,77.610031,Apollo Pharmacy,Pharmacy,12.936331,77.614682
1,Ansal Krisna,Residential Building (Apartment / Condo),12.937972,77.610031,Shakti Medical,Pharmacy,12.936130,77.615041
2,Ansal Krisna,Residential Building (Apartment / Condo),12.937972,77.610031,Sagar Medicals,Pharmacy,12.930985,77.607674
3,Ansal Krisna,Residential Building (Apartment / Condo),12.937972,77.610031,Rwl Chemist & Druggist,Pharmacy,12.936356,77.614772
0,Victory Ashoka Enclave,Residential Building (Apartment / Condo),13.000334,77.611456,Apollo Pharmacy,Pharmacy,13.001382,77.612495


In [30]:
# set the radius and limit
radius_b = 1000 # 1 kms range 
LIMIT_b = 5 # top 5 nearby Bus_Stop

# function to get nearby Bus_Stop for a spot
def get_nearby_Bus_Stop_for_a_spot(latitude, longitude):

    spot_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius_sm, LIMIT_sm, category_Bus_Stop)

    results = requests.get(spot_url).json()

    # return the venues for the provided category
    return get_nearby_venues(results)

In [31]:
# for each spot in Residence get the nearby Bus_Stop within 1 km
column_names = ["location", "location_category", "location_lat", "location_lng", "name", "categories", "lat", "lng"]

Bus_Stop_combined = pd.DataFrame(columns = column_names)
df = Residence
for lat, lng, name, category in zip(df['lat'], df['lng'], df['name'], df['categories']):
    # get the nearby Bus_Stop for a spot
    Bus_Stop = get_nearby_Bus_Stop_for_a_spot(lat, lng)

    if Bus_Stop.shape[0] > 0:
        Bus_Stop.insert(0, 'location', name)
        Bus_Stop.insert(1, 'location_category', category)
        Bus_Stop.insert(2, 'location_lat', lat)
        Bus_Stop.insert(3, 'location_lng', lng)
        
        if Bus_Stop.shape[0] == 1 :
            print("Only one Bus_Stop found near Residence:", name)
        if Bus_Stop.shape[0] == 2 :
            print("Only two Bus_Stop found near Residence:", name)
        if Bus_Stop.shape[0] == 3 :
            print("Only three Bus_Stop found near Residence:", name)
        if Bus_Stop.shape[0] == 4 :
            print("Only four Bus_Stop found near Residence:", name)
        if Bus_Stop.shape[0] > 4 :
            print("More than four Bus_Stop found near Residence:", name)

        # concat the Bus_Stop data to the combined dataframe
        Bus_Stop_combined = pd.concat([Bus_Stop_combined, Bus_Stop])
    else:
        print("NO Bus_Stop found within 1 KMS range near Residence:", name)

Bus_Stop_combined.reset_index(drop=True)
Bus_Stop_combined

Only two Bus_Stop found near Residence: Diamond District
Only one Bus_Stop found near Residence: Ansal Krisna
NO Bus_Stop found within 1 KMS range near Residence: Victory Ashoka Enclave
NO Bus_Stop found within 1 KMS range near Residence: Arya Vishwas
NO Bus_Stop found within 1 KMS range near Residence: Casa Nuestra
NO Bus_Stop found within 1 KMS range near Residence: Elita Promenade
Only one Bus_Stop found near Residence: 6th Block Rajaji Nagar
NO Bus_Stop found within 1 KMS range near Residence: Salarpuria Greenage
Only three Bus_Stop found near Residence: Brigade Gateway
NO Bus_Stop found within 1 KMS range near Residence: Brigade Gardenia, JP Nagar 7th Phase
NO Bus_Stop found within 1 KMS range near Residence: Pebble Bay
NO Bus_Stop found within 1 KMS range near Residence: L & T South city
NO Bus_Stop found within 1 KMS range near Residence: HM Winchester
Only one Bus_Stop found near Residence: Vars Topaz Apartments
NO Bus_Stop found within 1 KMS range near Residence: Trivandrum Lo

,location,location_category,location_lat,location_lng,name,categories,lat,lng
0,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,Old Airport Road Bus Stop,Bus Stop,12.959391,77.645580
1,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,Doopanahalli Bus Stop,Bus Stop,12.956775,77.649325
0,Ansal Krisna,Residential Building (Apartment / Condo),12.937972,77.610031,Madiwala Checkpost Bus Stop,Bus Stop,12.932704,77.612853
0,6th Block Rajaji Nagar,Residential Building (Apartment / Condo),12.979021,77.554819,Dhobi Ghat Bus Stop,Bus Stop,12.987228,77.550863
0,Brigade Gateway,Residential Building (Apartment / Condo),13.012168,77.556288,Yeshwantpr bus stand (Yeshwantpur Bus Stand),Bus Stop,13.019883,77.555759
1,Brigade Gateway,Residential Building (Apartment / Condo),13.012168,77.556288,Central Bus Stop,Bus Stop,13.006866,77.554472
2,Brigade Gateway,Residential Building (Apartment / Condo),13.012168,77.556288,SRS travels,Bus Stop,13.021604,77.551522
0,Vars Topaz Apartments,Residential Building (Apartment / Condo),12.939568,77.628193,durgama motors,Bus Stop,12.935039,77.624354
0,Sobha Dew Flower,Residential Building (Apartment / Condo),12.914919,77.578907,Sangam Circle,Bus Stop,12.917545,77.578609
1,Sobha Dew Flower,Residential Building (Apartment / Condo),12.914919,77.578907,Igate Sarakki Bus Stop,Bus Stop,12.906437,77.573905


In [32]:
# set the radius and limit
radius_c = 1000 # 1 kms range 
LIMIT_c = 5 # top 5 nearby colleges

# function to get nearby colleges for a spot
def get_nearby_colleges_for_a_spot(latitude, longitude):

    spot_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius_sm, LIMIT_sm, category_colleges)

    results = requests.get(spot_url).json()

    # return the venues for the provided category
    return get_nearby_venues(results)

In [33]:
# for each spot in Residence get the nearby colleges within 1 km
column_names = ["location", "location_category", "location_lat", "location_lng", "name", "categories", "lat", "lng"]

colleges_combined = pd.DataFrame(columns = column_names)
df = Residence
for lat, lng, name, category in zip(df['lat'], df['lng'], df['name'], df['categories']):
    # get the nearby colleges for a spot
    colleges = get_nearby_colleges_for_a_spot(lat, lng)

    if colleges.shape[0] > 0:
        # filter based on the category value containing University
        colleges = colleges[colleges['categories'].str.contains("University" )].reset_index(drop=True)
        colleges.insert(0, 'location', name)
        colleges.insert(1, 'location_category', category)
        colleges.insert(2, 'location_lat', lat)
        colleges.insert(3, 'location_lng', lng)
        
        if colleges.shape[0] == 1 :
            print("Only one colleges found near Residence:", name)
        if colleges.shape[0] == 2 :
            print("Only two colleges found near Residence:", name)
        if colleges.shape[0] == 3 :
            print("Only three colleges found near Residence:", name)
        if colleges.shape[0] == 4 :
            print("Only four colleges found near Residence:", name)
        if colleges.shape[0] > 4 :
            print("More than four colleges found near Residence:", name)

        # concat the colleges data to the combined dataframe
        colleges_combined = pd.concat([colleges_combined, colleges])
    else:
        print("NO colleges found within 1 KMS range near Residence:", name)

colleges_combined.reset_index(drop=True)
colleges_combined

Only one colleges found near Residence: Diamond District
Only one colleges found near Residence: Ansal Krisna
Only one colleges found near Residence: Casa Nuestra
Only one colleges found near Residence: Brigade Gateway
Only one colleges found near Residence: Pebble Bay
Only one colleges found near Residence: L & T South city
Only two colleges found near Residence: HM Winchester
Only two colleges found near Residence: Vars Topaz Apartments
Only one colleges found near Residence: Trivandrum Lodge
Only one colleges found near Residence: Golden Park Apartments
Only two colleges found near Residence: Prestige Falcon City
Only one colleges found near Residence: Mahakali Layout
Only one colleges found near Residence: Kormangala 8Th Block
Only one colleges found near Residence: 11th Cross First Block Rajajinagar
NO colleges found within 1 KMS range near Residence: HM World City
Only one colleges found near Residence: MSR Nagar
Only one colleges found near Residence: Esteem Park Apartments
Only

,location,location_category,location_lat,location_lng,name,categories,lat,lng
0,Diamond District,Residential Building (Apartment / Condo),12.960152,77.643188,arena animation,General College & University,12.964208,77.640892
0,Ansal Krisna,Residential Building (Apartment / Condo),12.937972,77.610031,Christ University,University,12.935617,77.606306
0,Casa Nuestra,Residential Building (Apartment / Condo),12.878629,77.579239,Maruthi school,General College & University,12.882973,77.575574
0,Brigade Gateway,Residential Building (Apartment / Condo),13.012168,77.556288,Utl,College & University,13.015733,77.548264
0,Pebble Bay,Residential Building (Apartment / Condo),13.044777,77.571413,Global Institute Of Corporate Education,General College & University,13.043710,77.570381
0,L & T South city,Residential Building (Apartment / Condo),12.886943,77.590236,Christ University Bannerghatta,General College & University,12.877988,77.595536
0,HM Winchester,Residential Building (Apartment / Condo),12.973311,77.605098,INIFD,General College & University,12.974693,77.605919
1,HM Winchester,Residential Building (Apartment / Condo),12.973311,77.605098,J T S Institute,General College & University,12.981077,77.601791
0,Vars Topaz Apartments,Residential Building (Apartment / Condo),12.939568,77.628193,Alchemy Testing Center - Prometric,General College & University,12.934699,77.629563
1,Vars Topaz Apartments,Residential Building (Apartment / Condo),12.939568,77.628193,"National Public School, Koramangala",General College & University,12.946111,77.622985


In [34]:
#counting the number schools near each residence within 1km range
schools_count=schools_combined
schools_count = schools_count.groupby('location').size().reset_index(drop=False)
schools_count.rename(columns={0: "count"}, inplace=True)
schools_count.sort_values('count', inplace=True)
type(schools_count)

schools_count.reset_index(drop=True)

,location,count
0,HSR Layout,1
1,Sobha Dew Flower,1
2,RMV Clusters,1
3,Purvi Lotus,1
4,Pebble Bay,1
5,MSR Nagar,1
6,Trivandrum Lodge,1
7,Victory Ashoka Enclave,1
8,Diamond District,1
9,"Brigade Gardenia, JP Nagar 7th Phase",1


In [35]:
#counting the number schools near each restraunts within 1km range

food_count=food_combined
food_count = food_count.groupby('location').size().reset_index(drop=False)
food_count.rename(columns={0: "count"}, inplace=True)
food_count.sort_values('count', inplace=True)
type(food_count)

food_count.reset_index(drop=True)


,location,count
0,Magan Silver Hill Apartments,1
1,Palace Cross Road,1
2,Kormangala 8Th Block,1
3,HM World City,1
4,Brigade Gateway,1
5,Platinum Lifestyle,1
6,Ansal Krisna,1
7,Khb Platinum,2
8,6th Block Rajaji Nagar,2
9,Prestige South Ridge,2


In [36]:
#counting the number schools near each hospitals within 1km range

hospitals_count=hospitals_combined
hospitals_count = hospitals_count.groupby('location').size().reset_index(drop=False)
hospitals_count.rename(columns={0: "count"}, inplace=True)
hospitals_count.sort_values('count', inplace=True)
type(hospitals_count)

hospitals_count.reset_index(drop=True)

,location,count
0,Legend Ornate,1
1,Perody MVB County,2
2,Axis Aspira,2
3,Casa Nuestra,2
4,RMV Clusters,2
5,HM Winchester,3
6,Ranka Enclave,3
7,Mantri Serenity,3
8,Mahaveer Lakes,3
9,Vars Topaz Apartments,3


In [37]:
#counting the number schools near each Pharmacy within 1km range

Pharmacy_count=Pharmacy_combined
Pharmacy_count = Pharmacy_count.groupby('location').size().reset_index(drop=False)
Pharmacy_count.rename(columns={0: "count"}, inplace=True)
Pharmacy_count.sort_values('count', inplace=True)
type(Pharmacy_count)

Pharmacy_count.reset_index(drop=True)


,location,count
0,Salarpuria Greenage,1
1,Prestige South Ridge,1
2,Siri Crystal,1
3,Palace Cross Road,1
4,Mahaveer Calyx,1
5,Legend Ornate,1
6,Khb Platinum,1
7,Casa Nuestra,2
8,Magan Silver Hill Apartments,2
9,Renaissance Temple Bells,2


In [38]:
#counting the number schools near each Shopping_Mall within 1km range

Shopping_Mall_count=Shopping_Mall_combined
Shopping_Mall_count = Shopping_Mall_count.groupby('location').size().reset_index(drop=False)
Shopping_Mall_count.rename(columns={0: "count"}, inplace=True)
Shopping_Mall_count.sort_values('count', inplace=True)
type(Shopping_Mall_count)

Shopping_Mall_count.reset_index(drop=True)

,location,count
0,MSR Nagar,1
1,Ali Apartments,1
2,Elita Promenade,1
3,Sobha Dew Flower,1
4,Salarpuria Greenage,1
5,Arya Vishwas,2
6,L & T South city,2
7,Prestige South Ridge,2
8,Prestige Falcon City,2
9,Trivandrum Lodge,3


In [39]:
#counting the number schools near each Bus_Stop within 1km range

Bus_Stop_count=Bus_Stop_combined
Bus_Stop_count = Bus_Stop_count.groupby('location').size().reset_index(drop=False)
Bus_Stop_count.rename(columns={0: "count"}, inplace=True)
Bus_Stop_count.sort_values('count', inplace=True)
type(Bus_Stop_count)

Bus_Stop_count.reset_index(drop=True)

,location,count
0,11th Cross First Block Rajajinagar,1
1,6th Block Rajaji Nagar,1
2,Ansal Krisna,1
3,Kormangala 8Th Block,1
4,Prestige South Ridge,1
5,Vars Topaz Apartments,1
6,Diamond District,2
7,Esteem Park Apartments,2
8,Mantri Terrace,2
9,Sobha Dew Flower,2


In [40]:
#counting the number schools near each colleges within 1km range

colleges_count=colleges_combined
colleges_count = colleges_count.groupby('location').size().reset_index(drop=False)
colleges_count.rename(columns={0: "count"}, inplace=True)
colleges_count.sort_values('count', inplace=True)
type(colleges_count)

colleges_count.reset_index(drop=True)

,location,count
0,11th Cross First Block Rajajinagar,1
1,Trivandrum Lodge,1
2,The Essence Suites,1
3,Surabhi Appts.,1
4,Renaissance Temple Bells,1
5,RMV Clusters,1
6,Pebble Bay,1
7,Palace Cross Road,1
8,Mantri Terrace,1
9,Mahakali Layout,1


In [41]:
df1 =food_count.reset_index(drop=True)
df2 =Bus_Stop_count.reset_index(drop=True)
df3 =schools_count.reset_index(drop=True)
df4 =colleges_count.reset_index(drop=True)
df5 =hospitals_count.reset_index(drop=True)
df6 =Pharmacy_count.reset_index(drop=True)
df7 =Shopping_Mall_count.reset_index(drop=True)

In [42]:
#combined list resicence with all nearby places
final_df1 = df1.merge(df2, how='left', left_on='location', right_on='location')
final_df1.rename(columns={"count_x": "Number of Restraunts", "count_y":"Number of Bus stops",},inplace=True)
final_df2 = final_df1.merge(df3, how='left', left_on='location', right_on='location').merge(df4, how='left', left_on='location', right_on='location').merge(df6, how='left', left_on='location', right_on='location')
final_df2.rename(columns={"count_x": "Number of schools", "count_y":"Number of Colleges","count":"Number of hospitals"},inplace=True)
final_df3 = final_df2.merge(df5, how='left', left_on='location', right_on='location').merge(df6, how='left', left_on='location', right_on='location')
final_df3.rename(columns={"count_x": "Number of Pharmacy", "count_y":"Number of Shopping Malls"},inplace=True)
#final_df4=final_df3
final_df3


,location,Number of Restraunts,Number of Bus stops,Number of schools,Number of Colleges,Number of hospitals,Number of Pharmacy,Number of Shopping Malls
0,Magan Silver Hill Apartments,1,NaN,NaN,NaN,2.0,5.0,2.0
1,Palace Cross Road,1,NaN,NaN,1.0,1.0,5.0,1.0
2,Kormangala 8Th Block,1,1.0,NaN,1.0,5.0,4.0,5.0
3,HM World City,1,NaN,NaN,NaN,NaN,NaN,NaN
4,Brigade Gateway,1,3.0,3.0,1.0,3.0,5.0,3.0
5,Platinum Lifestyle,1,NaN,NaN,NaN,NaN,NaN,NaN
6,Ansal Krisna,1,1.0,NaN,1.0,4.0,5.0,4.0
7,Khb Platinum,2,NaN,NaN,1.0,1.0,NaN,1.0
8,6th Block Rajaji Nagar,2,1.0,2.0,NaN,3.0,5.0,3.0
9,Prestige South Ridge,2,1.0,NaN,NaN,1.0,5.0,1.0


In [43]:
#final list of resicence/location which are having all the nearby places within 1km range
final_df = final_df3
final_df.dropna(how='any',axis=0,inplace=True)
final_df.reset_index(drop=True,inplace=True)
final_df

,location,Number of Restraunts,Number of Bus stops,Number of schools,Number of Colleges,Number of hospitals,Number of Pharmacy,Number of Shopping Malls
0,Brigade Gateway,1,3.0,3.0,1.0,3.0,5.0,3.0
1,Renaissance Temple Bells,2,4.0,4.0,1.0,2.0,5.0,2.0
2,Esteem Park Apartments,2,2.0,2.0,1.0,4.0,3.0,4.0
3,Diamond District,2,2.0,1.0,1.0,5.0,5.0,5.0
4,Surabhi Appts.,3,2.0,3.0,1.0,3.0,4.0,3.0
5,11th Cross First Block Rajajinagar,3,1.0,3.0,1.0,3.0,5.0,3.0
6,Mantri Terrace,3,2.0,3.0,1.0,3.0,4.0,3.0


From the above result it’s clear that within 1km distance:
Total of 7 locations are having all the basic necessary amenities within 1km range. These locations are one of the most ideal places to invest or build a residential Apartments nearby this location based on the criteria we have defined in the project.

### 5.	Result and Discussion
Our overall data analysis shows that Bangalore has many residential places with all the basic amenities nearby. To identify the optimal location, we first identified the locations for restaurants, schools, colleges, bus stop, shopping malls, pharmacy and Hospitals category within the 10 KM range from Bangalore center point. We wanted to get the optimal location within 5KM city radius. Then filtered these data to get top 5 locations for each category.
We could have opted for 10 locations for each category. But with 10 locations for each category, we may be moving away from the city center point. To be near the city center point, I preferred to select the top 5 locations.
Once we got top 5 locations for each category, we got the nearby existing residences/locations for each of these categories using the Foursquare API, which are within 1km of distance from location.

The reason for considering 1km as the radius limit is, we wanted to look for a
residence within ideal distance, considering all the necessary category within the mentioned radius.
Then prepared a table to list out the location name and nearby residence. We got 6 such location with all the basic necessary locations nearby.


Recommendations: Based on the obtained result, i would recommend following locations/residential Apartments:
1.	Brigade Gateway
2.	Renaissance Temple Bells
3.	Esteem Park Apartments	
4.	Diamond District	    
5.	Surabhi Appts      
6.  11th Cross First Block Rajajinagar	          
7.	Mantri Terrace	


### 6.	Conclusion
Purpose of this project is to identify the optimal location for investing/finding residential Apartment/location within 5KM range of Bangalore city. This would help the stakeholders/individuals in narrowing

down the search for location. Initially, I identified the top 5 locations for different categories. These categories normally attract individuals/builders. Hence scope for finding an optimal location/Residential Apartment near such location is more. Then we identified the nearby such locations/ Residential Apartment.

My finding and recommendations are already discussed in the Result and Discussion section. Please check the recommendations. Selecting the optimal location is left to the stakeholder based on the other dependent factor such as rent, water availability, parking slots, connectivity etc. It is up to the stakeholder who must pick one of the recommended locations.

